# Workflow and Task Options

Options for workflow can be set via "options" classes.

There are two categories of options provided in the library:

* **TaskOptions**:
    These options are used to set parameters for tasks in the workflow. An example of `TaskOptions` class is the `BaseExperimentOptions` which contains options for setting how experiments should run. For examples, the number of experiment shots `counts`, the averaging mode `averaging_mode`, the acquisition type `acquisition_type` and so on. 

* **WorkflowOptions**:
    These options are used to set parameters for the workflow. These include the settings for operating the workflow, such as `logstore` which specifies the protocol for storing a collection of records of workflow execution.
    In addition, `WorkflowOptions` contains options for the constituent tasks and sub-workflow. Users typically do not have to manually set these options as they are automatically generated by the workflow.

In [ ]:
from __future__ import annotations

from laboneq import workflow

# Create a new experiment options class
To create a new option class in your software, use the `@workflow.options` decorator.

You can define fields using the dataclass syntax like `x: int = 1` in the below cell.

For more tailored field definitions such as adding descriptions (docstrings), you can use `workflow.option_field` provided by our library. Check out the API reference of `workflow.option_field` for more details.

Ensure that attributes within the options class have default values. This setup allows the workflow to operate with those default settings when specific options aren't specified by the user.

In [ ]:
@workflow.options
class NewExperimentOptions:
    x: int = 1
    y: int = workflow.option_field(2, description="This is the y field")


opt = NewExperimentOptions()

If the options classes are defined without default values, an error will be raised when the options instance is created.

In [ ]:
@workflow.options
class InvalidExperimentOptions:
    no_default: int


opt = InvalidExperimentOptions(no_default=1)

You can always create a 'raw' options class like the above. However, the library provides a set of useful methods for options classes, such as nice printing and serialization/deserialization to dictionaries.

To make use of these methods, inherit from the `workflow.WorkflowOptions` or `workflow.TaskOptions` classes.

In [ ]:
@workflow.options
class NewExperimentOptions(workflow.TaskOptions):
    x: int = 1
    y: int = workflow.option_field(2, description="This is the y field")


opt = NewExperimentOptions()
opt

In [ ]:
opt.to_dict()

# Enable the options feature in the workflow

The advantage of using the options feature of a workflow is the automatic passing of options to the tasks and other features like run_until, etc.

At the moment, the options feature is enabled by default in the workflow when the arguments include `options`. In addition, the types for `options` must conform to a certain rule.

Let's illustrate the concepts via an example. 

`myworkflow` contains task `mytask` which takes in an argument and options. 

To use `myworkflow` with options, we must provide the right types for the `options` argument in `myworkflow`.

The supported types are:
- `WorkflowOptionsA | None = None`
- `Union[WorkflowOptionsA, None] = None`
- `Optional[WorkflowOptionsA]`

where `WorkflowOptionsA` is a subclass of `WorkflowOptions`.

From Python 3.10 onward, it is recommended to use `WorkflowOptionsA | None` to conform with the standard practice.

On Python 3.9, `from _future_ import annotations` must be imported to use `WorkflowOptionsA | None`.


In [ ]:
class WorkflowOptionsA(workflow.WorkflowOptions):
    count: int = workflow.option_field(1024, description="The number of repetitions")


@workflow.options
class TaskOptionsB(workflow.TaskOptions):
    count: int = workflow.option_field(1024, description="The number of repetitions")
    some_task_option: bool = workflow.option_field(
        False, description="Some task option."
    )

In [ ]:
@workflow.task
def task1(options: TaskOptionsB | None = None):
    options = options or TaskOptionsB()
    return options.count


@workflow.task
def task2(options: TaskOptionsB | None = None):
    options = options or TaskOptionsB()
    return options.count


@workflow.workflow
def inner_workflow(options: workflow.WorkflowOptions | None = None):
    task1()
    task2()

In [ ]:
@workflow.workflow
def myworkflow(options: WorkflowOptionsA | None = None):
    task1()
    task2()
    inner_workflow()

# Disallowed types for options

When the type provided for the options includes a subclass of `WorkflowOptions`, we assume that users are attempting to use the options feature. Hence, if the specified type does not follow the form of the above-mentioned types, an error will be raised to inform the user about this, as shown below. 

In [ ]:
# an error will be raised


@workflow.workflow
def invalid_workflow(options: WorkflowOptionsA | str):
    task1()
    task2()

# Run the workflow with options

We can create the options for a specific workflow directly by calling the `options()` method on the workflow. This returns an `OptionBuilder` instance, a wrapper of the `WorkflowOption` instance that contains the options for this specific workflow.

`OptionBuilder` helps to set the options fields more easily.

In [ ]:
workflow_opt = myworkflow.options()

The options contained in the workflow could be displayed by printing the option builder.

In [ ]:
workflow_opt

# Querying and setting the option fields

The fields of the workflow options together with the options of nested tasks and nested workflows are visible as attributes of the `OptionBuilder` instance.

In [ ]:
workflow.show_fields(workflow_opt)

To query the value of an option field, simply access it as an attribute of the `OptionBuilder` instance, as shown below for the case of field `count`. Here, we see that there are five `count` fields shown together with its values: one at the top-level of the options `(base,1024)` and others at the sub-task and sub-workflow levels.

In [ ]:
workflow_opt.count

To set all `count` fields,

In [ ]:
workflow_opt.count(2048)
workflow_opt.count

Standard array slicing is also supported.

In [ ]:
workflow_opt.count[1:](1024)
workflow_opt.count

For workflow options with more complicated structure, option fields can be set by specifying the task or workflow names. Let's start with setting top levels `count` to 0.

In [ ]:
workflow_opt.count(0, ".")
workflow_opt.count

Similarly, we can set all `count` that appear as top-level fields of `inner_workflow`.

In [ ]:
workflow_opt.count(1, "inner_workflow")
workflow_opt.count

To set a specific option fields, we need to specify its full path. Let's set `count` to 2 for the `task1` of the `inner_workflow` and `task1` at the top-level.

In [ ]:
workflow_opt.count(2, "task1")
workflow_opt.count(2, "inner_workflow.task1")
workflow_opt.count

Now to run the workflow with the modified options, simply pass the options to the workflow initialization

In [ ]:
res = myworkflow(options=workflow_opt).run()
res.tasks[1].output

# Standard options classes

In the [Applications Library package](https://github.com/zhinst/laboneq-applications) we provide a few standard options classes for quantum experiments and analyses, such as `BaseExperimentOptions`, `TuneupExperimentOptions` and `TuneUpWorkflowOptions`, `TuneupAnalysisOptions` and `TuneUpAnalysisWorkflowOptions`, etc. 

These implementations can be used in your workflows and tasks, or as a starting point for creating new options classes to serve your needs. 

Check out the Applications Library section of this manual to learn more!